In [ ]:
!pip install wandb -qU

In [ ]:
import wandb
wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

# Config testing

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
import pickle
import os
import copy
import sys
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score, roc_curve, auc
import wandb

In [ ]:
config = {
  "lr": 1e-3,
  "weight_decay": 1e-4,
  "epoch_num": 100
}
tags = ["new_data", "10percent", "BERT-Pretrained"]
run = wandb.init(entity='sentimovie', project="Bert-finetune", config=config, tags=tags)

In [ ]:
#######################################
###           wandb logging         ###
#######################################
##           Log Some data           ##
#######################################
tmp = {
    "sample_data": ["I think it was an interesting and thought-provoking movie.", "It was a silly and silly movie."], 
    "sample_label": ["Fresh", "Rotten"]}
wandb.log(tmp)
#######################################
#######################################

In [ ]:
#######################################
###           wandb logging         ###
#######################################
##            Log data size          ##
#######################################
tmp = {
    "train_size": len(dataset_sizes['train']),
    "val_size": len(dataset_sizes['val']),
    "test_size": len(dataset_sizes['test']),
    }
wandb.log(tmp)
#######################################
#######################################

In [8]:
optimizer = optim.Adam(model.parameters(), lr=wandb.config['lr'], weight_decay=wandb.config['weight_decay'])

0.001

In [10]:
def deflog(text):
    print(f"[DEF-LOG] {text}")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, 
                dataset_sizes, device, config, num_epochs=25):
    since = time.time()
    history = {'train_loss': [], 'val_loss': [], 
                    'train_acc': [], 'val_acc': []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in tqdm(range(num_epochs)):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(float(epoch_acc.cpu().numpy()))

            # deflog(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #######################################
            ###           wandb logging         ###
            #######################################
            ##        Log the training flow       ##
            #######################################
            tmp = {f'{phase}_loss': epoch_loss, 
                    f'{phase}_acc': epoch_acc,
                    'lr': config['lr'],
                    'epoch': epoch}
            wandb.log(tmp)
            #######################################
            #######################################

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    deflog(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    deflog(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [ ]:
def save_results(model, history, base_path="output"):
    history_file_path = f"{base_path}/history.pkl"
    with open(history_file_path, 'wb') as handle:
        pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    deflog(f'history has been saved in \"{history_file_path}\"')

    model_file_path = f"{base_path}/model.pt"
    torch.save(model.state_dict(), model_file_path)
    deflog(f'model has been saved in \"{model_file_path}\"')

In [ ]:
model, history = train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, 
                             device, wandb.config, num_epochs=wandb.config['epoch_num'])

In [ ]:
def testing(model, dataloader, device):
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_pred, y_true

In [ ]:
y_pred, y_true = testing(model, dataloaders['test'], device)

f1 = f1_score(y_true, y_pred, average='macro')
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
AUC = auc(fpr, tpr)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
deflog("Metric calculated")
wandb.sklearn.plot_confusion_matrix(y_true, 
                                    y_pred, 
                                    ['NORM', 'TUM'])
deflog("Send confusion matrix")
tmp = {
    "test_acc": acc, 
    "test_f1": f1, 
    "test_auc": AUC, 
    "test_precision": precision, 
    "test_recall": recall
}
wandb.log(tmp)
deflog("Run finished")

# Hyper-parameter testing

In [ ]:
def run_model(config=None):
    with wandb.init(entity='sentimovie', project="testing", config=config):
        config_wandb = wandb.config
        print(config_wandb.lr)

In [ ]:
sweep_config = {'method': 'random'}
# sweep_config = {'method': 'grid'}
# sweep_config = {'method': 'bayes'}

metric = {'name': 'val_acc', 
        'goal': 'maximize'}

sweep_config['metric'] = metric

hyper_parameters = {
    'lr': { 'values': [0.1, 0.01, 0.001, 0.0001, 0.00001]
    }
}
sweep_config['parameters'] = hyper_parameters
sweep_id = wandb.sweep(sweep_config, project="testing")
wandb.agent(sweep_id, run_model, count=6)

In [ ]:
[float(f"3e-{i}") for i in range(2,7)]

[0.03, 0.003, 0.0003, 3e-05, 3e-06]

In [ ]:
[float(f"1e-{i}") for i in range(2,7)]

[0.01, 0.001, 0.0001, 1e-05, 1e-06]